In [1]:
pip install speechrecognition pyaudio pyttsx3 ipywidgets


     --------------------------------------- 32.9/32.9 MB 10.4 MB/s eta 0:00:00
     ------------------------------------- 164.1/164.1 kB 10.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import time
import speech_recognition as sr
from groq import Groq
import pyttsx3
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# Initialize Groq client
client = Groq(api_key="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")

# ----------- AVATAR DISPLAY -------------
def show_avatar(status="idle"):
    """Show emoji avatar based on status."""
    clear_output(wait=True)
    if status == "listening":
        avatar_html = "<div style='text-align:center;font-size:70px;'>🧏‍♂️<p>Listening...</p></div>"
    elif status == "thinking":
        avatar_html = "<div style='text-align:center;font-size:70px;'>🤔<p>Thinking...</p></div>"
    elif status == "talking":
        avatar_html = "<div style='text-align:center;font-size:70px;'>🗣️<p>Talking...</p></div>"
    else:
        avatar_html = "<div style='text-align:center;font-size:70px;'>🙂<p>Idle...</p></div>"
    display(HTML(avatar_html))

# ----------- SPEECH INPUT -------------
def listen_to_user():
    """Listen via microphone and return recognized text."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        show_avatar("listening")
        print("🎤 Speak now...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    try:
        text = recognizer.recognize_google(audio)
        print(f"🧑‍💻 You said: {text}")
        return text
    except sr.UnknownValueError:
        print("❌ Sorry, I didn't understand.")
        return None
    except sr.RequestError:
        print("⚠️ Speech Recognition service error.")
        return None

# ----------- ASK AI + TTS -------------
def ask_ai(question, use_voice=True):
    """Send question to Groq LLM and speak reply."""
    try:
        show_avatar("thinking")
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a kind, helpful avatar assistant."},
                {"role": "user", "content": question}
            ],
            model="llama-3.3-70b-versatile",
            temperature=0.7,
            max_tokens=512
        )
        answer = response.choices[0].message.content
        print(f"\n🤖 Avatar: {answer}\n")

        if use_voice:
            show_avatar("talking")
            tts = pyttsx3.init()
            tts.say(answer)
            tts.runAndWait()
            tts.stop()
        show_avatar("idle")
        return answer

    except Exception as e:
        print(f"Error: {e}")
        show_avatar("idle")
        return None

# ----------- MAIN CHAT LOOP -------------
def start_avatar_voice_chat():
    """Full loop: listen → understand → reply."""
    print("🎯 Voice Avatar Ready! Say something (or say 'quit' to stop)\n")
    show_avatar("idle")

    while True:
        user_text = listen_to_user()
        if not user_text:
            continue
        if user_text.lower() in ["quit", "exit", "stop"]:
            print("👋 Goodbye!")
            show_avatar("idle")
            break
        ask_ai(user_text, use_voice=True)

# Run in Jupyter
start_avatar_voice_chat()